In [7]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pickle
import xrd
from importlib import reload
reload(xrd)

<module 'xrd' from '/Users/vekeryk/Projects/master-project/workspace/modules/xrd.py'>

In [8]:
from __future__ import annotations
import argparse
import pickle
from pathlib import Path
import numpy as np
import torch
import time
import torch.nn as nn
import torch.nn.functional as F

In [9]:
RANGES = {
    "Dmax1": (0.002, 0.030),
    "D01": (0.0010, 0.030),
    # "L1": (1000.0, 7000.0),
    # "Rp1": (0.0, 7000.0),
    "L1": (1000e-8, 7000e-8),
    "Rp1": (0.0, 7000e-8),
    "D02": (0.0020, 0.0300),
    # "L2": (1000.0, 7000.0),
    # "Rp2": (-6000.0, 0.0),
    "L2": (1000e-8, 7000e-8),
    "Rp2": (-6000e-8, 0.0),
}
PARAM_NAMES = list(RANGES.keys())

In [10]:
class ResidualBlock(nn.Module):
    def __init__(self, c: int, dilation: int = 1):
        super().__init__()
        pad = dilation * 3
        self.conv1 = nn.Conv1d(c, c, kernel_size=7,
                               padding=pad, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(c)
        self.conv2 = nn.Conv1d(c, c, kernel_size=7,
                               padding=pad, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(c)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        h = self.act(self.bn1(self.conv1(x)))
        h = self.bn2(self.conv2(h))
        return self.act(x + h)

In [11]:
class XRDRegressor(nn.Module):
    def __init__(self, n_out: int | None = None):
        super().__init__()
        if n_out is None:
            n_out = len(PARAM_NAMES)
        self.stem = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=9, padding=4),
            nn.BatchNorm1d(32),
            nn.ReLU(inplace=True),
        )
        self.blocks = nn.Sequential(
            ResidualBlock(32, dilation=1),
            ResidualBlock(32, dilation=2),
            ResidualBlock(32, dilation=4),
            ResidualBlock(32, dilation=8),
        )
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
            nn.Linear(32, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, n_out),
            nn.Sigmoid(),  # outputs in [0,1]
        )

    def forward(self, x):
        x = self.stem(x)
        x = self.blocks(x)
        return self.head(x)


In [ ]:
model = XRDRegressor()

print(len(list(model.parameters())))
for param in model.parameters():
    # print(param.shape)
    pass

# list(model.parameters())[0][0].shape, list(model.parameters())[0].shape
list(model.parameters())[0][0], list(model.parameters())[0]
list(model.parameters())[0].shape
# torch.Size([32, 1, 9])
# array with 32 elements
# each is an array with an array with9 elements

40


torch.Size([32, 1, 9])

In [36]:
len(model.state_dict().keys())

67

In [38]:
model.state_dict().keys()

odict_keys(['stem.0.weight', 'stem.0.bias', 'stem.1.weight', 'stem.1.bias', 'stem.1.running_mean', 'stem.1.running_var', 'stem.1.num_batches_tracked', 'blocks.0.conv1.weight', 'blocks.0.conv1.bias', 'blocks.0.bn1.weight', 'blocks.0.bn1.bias', 'blocks.0.bn1.running_mean', 'blocks.0.bn1.running_var', 'blocks.0.bn1.num_batches_tracked', 'blocks.0.conv2.weight', 'blocks.0.conv2.bias', 'blocks.0.bn2.weight', 'blocks.0.bn2.bias', 'blocks.0.bn2.running_mean', 'blocks.0.bn2.running_var', 'blocks.0.bn2.num_batches_tracked', 'blocks.1.conv1.weight', 'blocks.1.conv1.bias', 'blocks.1.bn1.weight', 'blocks.1.bn1.bias', 'blocks.1.bn1.running_mean', 'blocks.1.bn1.running_var', 'blocks.1.bn1.num_batches_tracked', 'blocks.1.conv2.weight', 'blocks.1.conv2.bias', 'blocks.1.bn2.weight', 'blocks.1.bn2.bias', 'blocks.1.bn2.running_mean', 'blocks.1.bn2.running_var', 'blocks.1.bn2.num_batches_tracked', 'blocks.2.conv1.weight', 'blocks.2.conv1.bias', 'blocks.2.bn1.weight', 'blocks.2.bn1.bias', 'blocks.2.bn1.run

In [37]:
model.state_dict()

OrderedDict([('stem.0.weight',
              tensor([[[ 0.2724,  0.1927, -0.1890, -0.1420, -0.0874,  0.0926, -0.1567,
                        -0.3093, -0.1406]],
              
                      [[-0.2913, -0.1809,  0.3324, -0.2580,  0.3326,  0.2046,  0.2196,
                        -0.2812, -0.2098]],
              
                      [[-0.2552,  0.0375,  0.2635, -0.2191, -0.0845,  0.2009, -0.1252,
                         0.2369,  0.2752]],
              
                      [[ 0.2491,  0.2284, -0.2490, -0.0600,  0.2023, -0.0392,  0.0499,
                         0.0431,  0.2341]],
              
                      [[ 0.0891,  0.2129,  0.3192, -0.1972, -0.0795,  0.0889,  0.1891,
                         0.0830, -0.1227]],
              
                      [[-0.3195,  0.1221, -0.1702, -0.1497,  0.1084,  0.2249, -0.2501,
                         0.2964, -0.2333]],
              
                      [[ 0.0135, -0.1964,  0.0316,  0.2092, -0.0441, -0.3333, -0.1817,
      

default (--lr=0.001)
MAE by parameter (abs / % of range / % of mean):
  Dmax1: 0.000683  |  2.44% rng  |  4.20% mean
  D01: 0.001186  |  4.09% rng  |  28.65% mean
  L1: 0.000004  |  6.12% rng  |  9.24% mean
  Rp1: 0.000002  |  2.89% rng  |  13.55% mean
  D02: 0.001804  |  6.44% rng  |  25.62% mean
  L2: 0.000005  |  7.78% rng  |  26.80% mean
  Rp2: 0.000014  |  23.85% rng  |  -47.65% mean

--lr=0.1 - не працює

--lr=0.01
MAE by parameter (abs / % of range / % of mean):
  Dmax1: 0.000989  |  3.53% rng  |  6.08% mean
  D01: 0.001289  |  4.45% rng  |  31.14% mean
  L1: 0.000007  |  12.11% rng  |  18.27% mean
  Rp1: 0.000003  |  3.99% rng  |  18.72% mean
  D02: 0.002020  |  7.22% rng  |  28.68% mean
  L2: 0.000007  |  11.18% rng  |  38.52% mean
  Rp2: 0.000015  |  24.64% rng  |  -49.23% mean

--lr=0.0001
MAE by parameter (abs / % of range / % of mean):
  Dmax1: 0.000853  |  3.05% rng  |  5.24% mean
  D01: 0.001322  |  4.56% rng  |  31.92% mean
  L1: 0.000004  |  6.71% rng  |  10.13% mean
  Rp1: 0.000003  |  4.21% rng  |  19.74% mean
  D02: 0.002371  |  8.47% rng  |  33.66% mean
  L2: 0.000006  |  10.21% rng  |  35.18% mean
  Rp2: 0.000015  |  24.77% rng  |  -49.50% mean

--lr=0.00001 (набагато повільніший прогрес, 80 епох)
MAE by parameter (abs / % of range / % of mean):
  Dmax1: 0.000856  |  3.06% rng  |  5.26% mean
  D01: 0.001360  |  4.69% rng  |  32.85% mean
  L1: 0.000004  |  7.38% rng  |  11.13% mean
  Rp1: 0.000004  |  5.77% rng  |  27.06% mean
  D02: 0.002442  |  8.72% rng  |  34.67% mean
  L2: 0.000006  |  10.78% rng  |  37.12% mean
  Rp2: 0.000015  |  24.82% rng  |  -49.60% mean